In [17]:
import pandas as pd
import numpy as np
import os
import glob
import json
from sharedcontrolpaper.force_sensitive_stopping_task_utils import get_subject_label, aggregate_trial_data, process_trial_data, grab_mean_metric, find_sum_of_intervals, convert_dict_to_df, calculate_proportions_non_nan, collect_trial_metric, grab_mean_metric_by_halves, convert_formats, rename_index_column, convert_to_milliseconds

In [18]:
parent_directory = os.path.dirname(os.getcwd())
data_path = os.path.join(parent_directory, 'data', 'experiment')
task = "force_sensitive_stopping_task"
exp_stage = "final"
pattern = os.path.join(data_path, exp_stage, '*', task, '*.csv')
data_files = glob.glob(pattern)

In [19]:
shared_control_metrics = {}
for file in data_files:
    subject_label = get_subject_label(file)
    df = pd.read_csv(file)
    df['block'] = df['block'].str.strip("'")
    df_test = df.query("block != 'practice'")
    block_1 = df_test.query("block == 'block 1'").reset_index(drop=True)
    block_2 = df_test.query("block == 'block 2'").reset_index(drop=True)

    task_dfs = [block_1, block_2]

    for df in task_dfs:
        if 'AI-assisted' in df['condition'].values:
            ai_data = df.copy()
            ai_data_agg = aggregate_trial_data(ai_data)
        else:
            non_ai_data = df.copy()
            non_ai_data_agg = aggregate_trial_data(non_ai_data)


    shared_control_metrics[subject_label] = {'AI': {'data': ai_data_agg}, 'Non-AI': {'data': non_ai_data_agg}}

    for block in shared_control_metrics[subject_label].keys():
        trial_results, ssrt_list = process_trial_data(shared_control_metrics[subject_label][block]['data'], block=block)
        shared_control_metrics[subject_label][block]['trial_results'] = trial_results
        shared_control_metrics[subject_label][block]['ssrt_list'] = ssrt_list

## Excluded Observations

## Grabbing SSRT and other metrics across conditions

In [20]:
force_sensitive_stopping_task_ssrt = grab_mean_metric(shared_control_metrics, 'ssrt')

In [21]:
duration_of_inhibition = grab_mean_metric(shared_control_metrics, 'duration_of_inhibition')

In [22]:
go_task_accuracy_before_stop_onset = grab_mean_metric(shared_control_metrics, 'go_task_accuracy_before_stop_onset')

In [23]:
go_task_accuracy_after_stop_onset = grab_mean_metric(shared_control_metrics, 'go_task_accuracy_after_stop_onset')

In [24]:
ball_after_ring_proportion_before_stop_onset = grab_mean_metric(shared_control_metrics, 'ball_after_ring_proportion_before_stop_onset')
print(ball_after_ring_proportion_before_stop_onset.mean())

non_ai         0.000177
ai_failed      0.000093
ai_assisted    0.000235
dtype: float64


In [25]:
first_non_zero_pressure_timestamp = grab_mean_metric(shared_control_metrics, 'first_non_zero_pressure_timestamp', aggregate_ai=True)
print(first_non_zero_pressure_timestamp.mean())

non_ai         0.050664
ai_failed      0.049287
ai_assisted    0.049287
dtype: float64


In [26]:
first_full_pressure_timestamp = grab_mean_metric(shared_control_metrics, 'first_full_pressure_timestamp', aggregate_ai=True)
print(first_full_pressure_timestamp.mean())

non_ai         0.177985
ai_failed      0.148956
ai_assisted    0.148956
dtype: float64


In [27]:
proportion_stops_before_stop_onset = grab_mean_metric(shared_control_metrics, 'proportion_stops_before_stop_onset')
print(proportion_stops_before_stop_onset.mean())

non_ai         0.025474
ai_failed      0.022171
ai_assisted    0.023202
dtype: float64


## Finding the proportion of full pressure points (pressure = 1) at each time interval

In [28]:
non_ai, ai_failed, ai_assisted = {}, {}, {}
for subject, subject_data in shared_control_metrics.items():
    results = collect_trial_metric(subject, subject_data, 'pressures_at_intervals_until_stop_onset')
    # Find the maximum interval length to pad the lists to the same length
    max_length = max(
        max([len(lst) for lst in results['non_ai']], default=0),
        max([len(lst) for lst in results['ai_failed']], default=0),
        max([len(lst) for lst in results['ai_assisted']], default=0)
    )
            
    non_ai = find_sum_of_intervals(results['non_ai'], non_ai, max_length, subject)
    ai_failed = find_sum_of_intervals(results['ai_failed'], ai_failed, max_length, subject)
    ai_assisted = find_sum_of_intervals(results['ai_assisted'], ai_assisted, max_length, subject)


# Convert dictionaries into DataFrames for each condition
time_intervals = [f"{i * 100}-{(i + 1) * 100}ms" for i in range(max_length)]

non_ai_proportion_ones, ai_failed_proportion_ones, ai_assisted_proportion_ones = (convert_dict_to_df(non_ai, time_intervals), 
                                                           convert_dict_to_df(ai_failed, time_intervals), 
                                                           convert_dict_to_df(ai_assisted, time_intervals))

/Users/kritiaxh/Documents/paperRepos/SharedControlPaper/src/sharedcontrolpaper/force_sensitive_stopping_task_utils.py:338: RuntimeWarning: invalid value encountered in divide
  measures_dict[subject] = np.nansum(np.vstack(trials) == 1, axis=0) / counts # Count number of pressures=1


## Finding the proportion of trials where subjects inhibited

In [29]:
condition_measure = {}
proportion = {}

for subject, subject_data in shared_control_metrics.items():
    results = collect_trial_metric(subject, subject_data, 'ssrt')
    counts, total_counts = calculate_proportions_non_nan(results)

    proportion[subject] = {
        'non_ai': counts['non_ai'] / total_counts['non_ai'] if total_counts['non_ai'] > 0 else 0,
        'ai_failed': counts['ai_failed'] / total_counts['ai_failed'] if total_counts['ai_failed'] > 0 else 0,
        'ai_assisted': counts['ai_assisted'] / total_counts['ai_assisted'] if total_counts['ai_assisted'] > 0 else 0,
    }


df = pd.DataFrame(proportion).T
df = df.sort_index()
print(df.mean())


non_ai         0.999750
ai_failed      0.998750
ai_assisted    0.989688
dtype: float64


## Create CSVs of SSRT by each half of trials in a block 

In [30]:
ssrt_first_half, ssrt_second_half = grab_mean_metric_by_halves(shared_control_metrics, 'ssrt')

## Store all data that should be accessed later into a JSON file

In [31]:
converted_shared_control_metrics = convert_formats(shared_control_metrics)
rename_index_column(force_sensitive_stopping_task_ssrt)
rename_index_column(duration_of_inhibition)

convert_to_milliseconds(force_sensitive_stopping_task_ssrt)
convert_to_milliseconds(duration_of_inhibition)
convert_to_milliseconds(ssrt_first_half)
convert_to_milliseconds(ssrt_second_half)

In [32]:
data_to_save = {
    'non_ai_proportion_ones': non_ai_proportion_ones.to_dict(),
    'ai_failed_proportion_ones': ai_failed_proportion_ones.to_dict(),
    'ai_assisted_proportion_ones': ai_assisted_proportion_ones.to_dict(),
    'ssrt_first_half': ssrt_first_half.to_dict(),
    'ssrt_second_half': ssrt_second_half.to_dict(),
    'go_task_accuracy_after_stop_onset': go_task_accuracy_after_stop_onset.to_dict(),
    'go_task_accuracy_before_stop_onset': go_task_accuracy_before_stop_onset.to_dict(),
    'duration_of_inhibition': duration_of_inhibition.to_dict(),
    'force_sensitive_stopping_task_ssrt': force_sensitive_stopping_task_ssrt.to_dict(),
    'shared_control_metrics': converted_shared_control_metrics
}

with open('force_sensitive_data.json', 'w') as f:
    json.dump(data_to_save, f, indent=4)